In [29]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [30]:
l1 = keras.layers.Dense(units=1, input_shape=[1])
model = keras.Sequential([l1])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0,-1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict(np.array([10.0])))
print(f"Weights: {l1.get_weights()}")


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 15.8471
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 12.7126
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 10.2414
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 8.2923
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 6.7540

c:\Users\minso\Desktop\tinyml\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 6.7540
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.5390
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.5785
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.8183
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.2158
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.7374
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.3567
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.0531
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.8101
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.6149
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.4575
Epoch 16/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.3298
Epoch 17/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.2255
Epoch 18/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.1398
Epoch 19/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.0688
Epoch 20/500
1/1 ━━━━━━━━━━

In [38]:
# Save the model
export_dir = 'saved_model/1'
model.export(export_dir)

INFO:tensorflow:Assets written to: saved_model/1\assets


INFO:tensorflow:Assets written to: saved_model/1\assets


Saved artifact at 'saved_model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor_21')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2288191815504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2288191814160: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [40]:
# Convert the saved model into TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# Create the new model as a physical file
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

1124

In [41]:
# Without using an embedded system, we can test our model in our computer

# Load TFLite model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)


[{'name': 'serving_default_keras_tensor_21:0', 'index': 0, 'shape': array([1, 1]), 'shape_signature': array([-1,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 3, 'shape': array([1, 1]), 'shape_signature': array([-1,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [42]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)

# Convert our input into a tensor our tflite model can understand
interpreter.set_tensor(input_details[0]['index'], to_predict)

# Run the prediction
interpreter.invoke()

# Convert our tensor output into something we can understand
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.98146]]
